In [1]:
import pandas as pd
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader, random_split
from pycocotools.coco import COCO
from pytorch_lightning import LightningDataModule
from typing import List, Any, Tuple, Optional
from PIL import Image
from pathlib import Path

In [10]:
df = pd.read_csv('/home.stud/kuntluka/dataset/cars/data/train_solution_bounding_boxes.csv')
df

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [2]:
def xywh_to_xyxy(box):
    x, y, w, h = box
    return [x, y, x+w, y+h]

class COCODataset(Dataset):
    def __init__(self, root, annFile, transforms):
        super().__init__()
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transfroms = transforms
        self.category_names = {}
        for cat in self.coco.loadCats(self.coco.getCatIds()):
            self.category_names[cat['id']] = cat['name']

    def _load_image(self, id: int) -> Image.Image:
        path = self.coco.loadImgs(id)[0]["file_name"]
        return Image.open(Path(self.root)/path).convert("RGB"), path

    def _load_target(self, id: int) -> List[Any]:
        targets = self.coco.loadAnns(self.coco.getAnnIds(id))
        boxes = []
        labels = []
        for box in targets:
            boxes.append(xywh_to_xyxy(box["bbox"]))
            labels.append(box["category_id"])

        return {"boxes": torch.tensor(boxes) if len(boxes) != 0 else torch.empty((0, 4)),
                "labels": torch.tensor(labels).type(torch.int64)}

    def __getitem__(self, index):
        id = self.ids[index]
        img, path = self._load_image(id)
        target = self._load_target(id)
        if self.transfroms is not None:
            transformed = self.transfroms(image=np.array(
                img), bboxes=target["boxes"], class_labels=target["labels"])
            img = transformed['image']
            target["boxes"] = torch.tensor(transformed["bboxes"]) if len(
                transformed["bboxes"]) != 0 else torch.empty((0, 4))
            target["labels"] = torch.tensor(
                transformed["class_labels"]).type(torch.int64)

        return img, target, path

    def __len__(self) -> int:
        return len(self.ids)

In [12]:
data_root = "/home.stud/kuntluka/dataset/carries_dataset/images"
ann_file ="/home.stud/kuntluka/dataset/carries_dataset/annotations.json"
ds = COCODataset(data_root, ann_file, None)
ds[0]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


(<PIL.Image.Image image mode=RGB size=1068x847 at 0x7FBCDE5FAA60>,
 {'boxes': tensor([[129.7300, 295.1500, 151.2200, 318.4400],
          [424.6700, 526.9100, 434.3000, 556.9700],
          [295.8500, 517.7900, 310.1700, 545.4000],
          [281.6200, 506.4100, 295.9400, 540.6500]]),
  'labels': tensor([1, 1, 1, 1])},
 '1.png')

In [11]:
data =[[1,"foooo",np.nan], [np.nan, "foo", 1], [1, "f", np.nan]]
df = pd.DataFrame(data, columns=['integer', 'name', 'size'])
df

,integer,name,size
0,1.0,foooo,NaN
1,NaN,foo,1.0
2,1.0,f,NaN


In [15]:
from pathlib import Path
from PIL import Image
data = []
data_root = Path(data_root)
for img, ann, f_name in ds:
    with Image.open(data_root / f_name) as im:
        width, height = im.size

    if torch.numel(ann['labels']) == 0 :
        data.append([f_name, True, width, height, np.nan, np.nan, np.nan, np.nan])
    else:
        for box in ann['boxes']:
            x1 = float(box[0])
            y1 = float(box[1])
            x2 = float(box[2])
            y2 = float(box[3])
            data.append([f_name, False, width, height, x1, y1, x2, y2])


df = pd.DataFrame(data, columns=['image', 'noobject', 'image_width', 'image_height', 'xmin', 'ymin', 'xmax', 'ymax'])
df.head()


,image,noobject,image_width,image_height,xmin,ymin,xmax,ymax
0,1.png,False,1068,847,129.729996,295.149994,151.220001,318.440002
1,1.png,False,1068,847,424.670013,526.909973,434.299988,556.969971
2,1.png,False,1068,847,295.850006,517.789978,310.170013,545.400024
3,1.png,False,1068,847,281.619995,506.410004,295.940002,540.650024
4,10.png,False,1068,847,464.000000,315.690002,475.179993,334.179993


In [16]:
df.to_csv("/home.stud/kuntluka/dataset/carries_dataset/annotations.csv", index = False)

In [15]:
from pathlib import Path
from PIL import Image
data_root = Path("/home.stud/kuntluka/dataset/cars/data/training_images")

h = []
w = []
for _,row in df.iterrows():
    # print(df.image)
    with Image.open(data_root / row.image) as im:
        width, height = im.size
        h.append(height)
        w.append(width)
df['image_width'] = w
df['image_height'] = h



In [17]:
df.to_csv('/home.stud/kuntluka/dataset/cars/data/train_solution_bounding_boxes.csv', index=False)